In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [2]:
ratings = {}
k = 28
home_advantage = 22
season_home = 0
season_away = 0
total_home = 0
total_away = 0

In [3]:
ratings['현대건설'] = 1500
ratings['GS칼텍스'] = 1500
ratings['KGC인삼공사'] = 1500
ratings['한국도로공사'] = 1500
ratings['흥국생명'] = 1500

In [4]:
for season in range(1,19):
    season_home = 0
    season_away = 0
    if season == 8:
        ratings['IBK기업은행'] = 1500
    if season == 18:
        ratings['페퍼저축은행'] = 1500
    if season <10:
        season = "0"+str(season)
    for game_number in range(1,300):
        url = "https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=0"+str(season)+"&g_part=201&r_round=1&g_num="+str(game_number)+"&"
        res = requests.get(url)
        print(url)
        soup = BeautifulSoup(res.text,"lxml")
        team_names = soup.find_all("span",attrs={"class":"team"}) #팀 이름을 리스트로 가져옴
        if team_names == []: #시즌이 끝나면 레이팅을 조정함
            for key in ratings:
                if ratings[key] != 0:
                    ratings[key] = ratings[key]+(1500-ratings[key])/2
            break
        match_date = soup.find("table", attrs={"class" : "lst_recentgame lst_result mt10"}).find("thead").find("tr").find("th").get_text()[1:21]
        home_team_name = team_names[0].get_text()
        away_team_name = team_names[1].get_text()
        if home_team_name == "도로공사":
            home_team_name = "한국도로공사"
        if away_team_name == "도로공사":
            away_team_name = "한국도로공사"
        if home_team_name == "KT&G":
            home_team_name = "KGC인삼공사"
        if away_team_name == "KT&G":
            away_team_name = "KGC인삼공사"
        if home_team_name == "인삼공사":
            home_team_name = "KGC인삼공사"
        if away_team_name == "인삼공사":
            away_team_name = "KGC인삼공사"
        if home_team_name not in ratings:
            continue
        if away_team_name not in ratings:
            continue
        results = soup.find_all("p",attrs={"class":re.compile("^result")}) #승패 결과를 리스트로 가져옴
        points = soup.find_all("p",attrs={"class":"num"}) #세트 스코어를 리스트로 가져옴
        if points == []: #게임이 없으면 시즌을 종료함
            break
        if season == 18: #18시즌이 끝나면 프로그램을 종료함
            if points[0].get_text() == points[1].get_text():
                break
        #모든 확인이 끝난 지점
        teams = soup.find_all("div",{"class":"wrp_lst"})
        for team in teams:
            data = []
            record = team.find("table",{"class":"lst_board lst_scroll w837 record_1"})
        expected_score = 1/(1+10**((ratings[away_team_name]-ratings[home_team_name]-home_advantage)/400)) #예상점수
        if int(points[0].get_text()) > int(points[1].get_text()): #홈팀 승리시 레이팅 조정
            total_home += 1
            season_home += 1
            ratings[home_team_name] = ratings[home_team_name]+k*(1-expected_score)
            ratings[away_team_name] = ratings[away_team_name]-k*(1-expected_score)
        else: #홈팀 패배시 레이팅 조정
            ratings[home_team_name] = ratings[home_team_name]+k*(0-expected_score)
            ratings[away_team_name] = ratings[away_team_name]-k*(0-expected_score)
            total_away += 1
            season_away += 1
    print(season_home,season_away,season_home/(season_home+season_away)*100)

https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=1&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=2&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=3&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=4&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=5&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=6&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=7&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=8&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=9&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1

In [5]:
sorted_dict = sorted(ratings.items(), key = lambda item: item[0], reverse = True)
print(sorted_dict)

[('흥국생명', 1451.5475454363038), ('현대건설', 1654.982010887099), ('한국도로공사', 1614.2731484954634), ('페퍼저축은행', 1322.5645450569532), ('KGC인삼공사', 1529.715734704549), ('IBK기업은행', 1370.9626522728477), ('GS칼텍스', 1555.9543631467843)]


In [6]:
def xwin_rate(A,B):
    return 1/(1+10**((B-A-22)/400))

In [29]:
xwin_rate(ratings['현대건설'], ratings['KGC인삼공사'])

0.700091335796886

In [31]:
1/1.19

0.8403361344537815